In [ ]:
import cv2
from numpy import * 


In [ ]:
img1 = cv2.imread('1_1.jpg') 
img2 = cv2.imread('1_2.jpg')
img1  = cv2.resize(img1,(0,0),fx = 0.2,fy = 0.2)
img2  = cv2.resize(img2,(0,0),fx = 0.2,fy = 0.2)
print img1.shape

In [ ]:
gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

In [ ]:
sift = cv2.xfeatures2d.SIFT_create()

In [ ]:
def calc_norm(arr1, arr2): 
    return linalg.norm(arr1-arr2)

In [ ]:
#Fundamental matrix from img1 to img2
def Fundamental(img1,img2):
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    good = []
    pts1 = []
    pts2 = []

    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = int32(pts1)
    pts2 = int32(pts2)
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    
    return F


In [ ]:
def epiline(pt2,F): 
    lines1 = cv2.computeCorrespondEpilines(pt2.reshape(-1,1,2), 2,F)
    return lines1
    

In [ ]:
def createGaussianFilter(size,stdv):
    if(size%2 == 1):
        gaussian = zeros([size,size])
        head = 1
        mid = (size-1)/2
        gaussian[mid,mid] = 1
        for i in range(0,size-mid): 
            for j in range(0,size-mid):
                power = (-i*i-j*j)/(2.*(stdv**2))
                gaussian[mid+i,mid+j] = gaussian[mid,mid]*exp(power)
                gaussian[mid-i,mid+j] = gaussian[mid,mid]*exp(power)
                gaussian[mid+i,mid-j] = gaussian[mid,mid]*exp(power)
                gaussian[mid-i,mid-j] = gaussian[mid,mid]*exp(power)
        
    if(size%2 == 0): 
        gaussian = zeros([size,size])
        head = 1
        mid = size/2
        gaussian[mid,mid] = 1
        for i in range(-size/2,size/2): 
            for j in range(-size/2,size/2): 
                power = (-i*i - j*j)/(2.*(stdv**2))
                gaussian[mid+i,mid+j] = gaussian[mid,mid]*exp(power)        
        
    normalized =  gaussian/sum(gaussian)
    return normalized

In [ ]:
def allpoints(img1): 
    rows,cols = img1.shape[0],img1.shape[1]
    points = []
    for i in range(rows):
        for j in range(cols): 
            points.append([j,i])
    return asarray(points)
            

In [ ]:
def createGaussianFilter(size,stdv):
    if(size%2 == 1):
        gaussian = zeros([size,size])
        head = 1
        mid = (size-1)/2
        gaussian[mid,mid] = 1
        for i in range(0,size-mid): 
            for j in range(0,size-mid):
                power = (-i*i-j*j)/(2.*(stdv**2))
                gaussian[mid+i,mid+j] = gaussian[mid,mid]*exp(power)
                gaussian[mid-i,mid+j] = gaussian[mid,mid]*exp(power)
                gaussian[mid+i,mid-j] = gaussian[mid,mid]*exp(power)
                gaussian[mid-i,mid-j] = gaussian[mid,mid]*exp(power)
        
    if(size%2 == 0): 
        gaussian = zeros([size,size])
        head = 1
        mid = size/2
        gaussian[mid,mid] = 1
        for i in range(-size/2,size/2): 
            for j in range(-size/2,size/2): 
                power = (-i*i - j*j)/(2.*(stdv**2))
                gaussian[mid+i,mid+j] = gaussian[mid,mid]*exp(power)        
        
    normalized =  gaussian/sum(gaussian)
    return normalized

In [ ]:
F = Fundamental(img1,img2)
pts = allpoints(img2)
lines = epiline(pts,F)
rows,cols = img2.shape[0],img2.shape[1]
final = zeros([rows,cols,3])
gaussi =  createGaussianFilter(4,4/6.)
gauss = zeros([4,4,3])
gauss[:,:,0] = gaussi
gauss[:,:,1] = gaussi
gauss[:,:,2] = gaussi
for i in range(len(pts)):
    if(i%1000==0):
        print i
    pt = pts[i]
    r = lines[i][0]
    x0 = 2
    x1 = cols-2
    if(pt[1]>=2 and pt[1]<rows-2 and pt[0]>=2 and pt[0]<cols-2):
        neigh1 = img2[pt[1]-2:pt[1]+2,pt[0]-2:pt[0]+2,:]
        neigh1 = gauss*neigh1
        neigh1 = neigh1.reshape(48)
        mini = 1000000
        p,q = 0,0
        for x in range(x0,x1,1):
            y = -int((r[2]+r[0]*x)/r[1])
            if(y>=2 and y<rows-2): 
                neigh2 = img1[y-2:y+2,x-2:x+2,:]
                neigh2 = gauss*neigh2
                neigh2 = neigh2.reshape(48)
                norm = calc_norm(neigh1,neigh2) 
                if(norm<mini): 
                    mini = norm
                    p,q = x,y

        final[pt[1],pt[0]] = img1[q,p]


In [ ]:
cv2.imshow('final',final.astype('uint8'))
cv2.waitKey(0)